In [1]:
import torch
import torch.nn as nn 

def corr2d(x,k):
    h,w = k.shape

    y = torch.zeros((x.shape[0] - h + 1,x.shape[1] - w + 1))
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            y[i,j] = (x[i:i + h ,j : j + w] * k).sum()

    return y

x = torch.tensor([[0,1,2],[3,4,5],[6,7,8]])
k = torch.tensor([[0,1],[2,3]])
corr2d(x,k)

tensor([[19., 25.],
        [37., 43.]])

In [2]:
class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        super(Conv2D,self).__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.randn(1))

    def forward(self,x):
        return corr2d(x,self.weight)  + self.bias

In [3]:
x = torch.ones(6,8)
x[:,2:6] = 0
x

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [4]:
k = torch.tensor([[1,-1]],dtype=torch.float)
y = corr2d(x,k)
y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [7]:

conv2d = Conv2D(kernel_size=(1,2))

step = 10
lr = 0.01

for i in range(step):
    y_hat = conv2d(x)
    l = ((y_hat - y) ** 2).sum()
    l.backward()

    conv2d.weight.data -= lr * conv2d.weight.grad
    conv2d.bias.data -= lr * conv2d.bias.grad

    # 梯度清0
    conv2d.weight.grad.fill_(0)
    conv2d.bias.grad.fill_(0)

    if (i + 1) % 5 == 0:
        print("step %d,loss %.3f" % (i + 1 ,l.item()))

print("weight:",conv2d.weight.data)
print("bias:",conv2d.bias.data)

step 5,loss 1.586
step 10,loss 0.307
weight: tensor([[ 0.8391, -0.9242]])
bias: tensor([0.0476])


In [11]:
def pool2d(x,pool_size,mode='max'):
    x = x.float()
    p_h,p_w = pool_size
    y = torch.zeros(x.shape[0] - p_h + 1,x.shape[1] - p_w + 1)
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            if mode == 'max':
                y[i,j] = x[i:i + p_h,j:j + p_w].max()
            elif mode == 'avg':
                y[i,j] = x[i:i + p_h,j : j + p_w].mean()
    return y

x = torch.tensor([[0,1,2],[3,4,5],[6,7,8]])
pool2d(x,(2,2))

tensor([[4., 5.],
        [7., 8.]])

In [12]:
pool2d(x,(2,2),'avg')

tensor([[2., 3.],
        [5., 6.]])